# Important Note

**In order to use these functions, you need to have separate, labeled images that have the cracks outlined in black.** This notebook can be utilized to score the thresholded crack images against ground truth images that have had the cracks traced over. Here are steps on how to use the code below:
1. Duplicate a folder of the segmented images (not yet thresholded)
2. For images with the cracks, trace over the cracks in black with an image editing feature of your choice
3. *Save in a .txt file the name of the pictures that you edited* - this will allow you to only test the accuracy of images with cracks since all the others without them are the same, edited or unedited.
4. Load the paths of the labeled segemented images and the .txt file that contains the name of the pictures into the `loadLabeledImages`. Save this function to a dictionary.
5. You can now load your dictionary from the step above and the dictionary of thresholded images into the `testImages` function and save them to a variable (i.e. `score`).
6. It is suggested to get one value rather than the score for each image, to print out the mean value of the variable from the above step (i.e. `print(numpy.average(list(score.values()))`)

# Install Dependencies

Uncomment and run the first time to make sure you have all the dependencies.

In [ ]:
#!pip install -r dependencies.txt
#!pip install opencv-contrib-python==4.5.3.56

# Imports

 Please run the following code to import all of the functions.

In [ ]:
import numpy as np
import os
from skimage import io
from skimage.filters import threshold_multiotsu
from src.CreatePaths import createPaths
from src.LoadImage import loadImage
from src.ThresholdCutImages import thresholdCutImages
from src.HeatImage import heatImage
from src.TestImages import loadLabeledImages, testImages
from src.SaveDictionary import saveDictionary
from src.StitchImageDict import stitchImageDict, stitchRows
import cv2

# Initialize Functions

Run the cells below to initialize the functions.

In [ ]:
def loadLabeledImages(path, images, color=(0,0,0)):
    """
    Loads in the the hand traced images and creates a binary image 
    of black pixels vs rest
    Args:
        path - the path to the folder containing ONLY the images - ground_truth folder.
        images - the path to the file containing the images labeled - .txt.
        color - the color the images were labeled in.
    """
    text = open(images,'r')
    labeled_images = [line for line in text.readlines() ]
    for name in range(len(labeled_images)):
        labeled_images[name] = labeled_images[name].strip()
    #print(labeled_images)
    
    labeled_image_dict = {}
    for file in os.listdir(path):
        if file[-8:-4] in labeled_images:
            key = (int(file[-8:-6]), int(file[-6:-4]))
            # some corrupted files so have to check
            try:
                # cv2 loads as BGR, not RGB
                # not applicable in this case
                img = cv2.imread(os.path.join(path, file))
                labeled_image_dict[key] = cv2.inRange(img[...,:3], color, color)
            except:
                #print('no good')
                print(key)
                
    return labeled_image_dict

In [ ]:
def testImages(labeled_image_dict, thresholded_dict):
    score_dict = {}
    for key in labeled_image_dict:
        # percent different
        # is a heuristic for percent wrong
        cLabeled = np.count_nonzero(labeled_image_dict[key])
        cThresh = np.count_nonzero(thresholded_dict[key])
        ans = np.min(cLabeled, cThresh) / np.max(cLabeled, cThresh)
        score_dict[key] = ans
    return score_dict

#### Example of what code may look like when using these functions

Step 4

In [ ]:
#labeled_img = loadLabeledImages('./data/ground_truth/', './data/LabeledImages.txt')

Step 5 + 6

In [ ]:
# score = testImages(labeled_img, image_dict[image]['thresholded_dict'])
# np.average(list(score.values()))